In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[3]:

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.cluster import KMeans

In [ ]:
# In[4]:


import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# In[5]:

In [ ]:
# In[55]:
path = 'C:/bigcon/'
import sys
sys.path.append(path + 'script')
from k_table import out, map_, df_res, df_act, df_res_act

In [ ]:
app_data = []

for i in sorted(glob(path + 'input/basic_dataset/*.csv')):
    app_data.append(pd.read_csv(i))
    
df_res = app_data[0] ## 주거자 데이터
df_act = app_data[1] ## 용인시에서 활동한 활동지 기준 데이터
df_res_act = app_data[2] ## 용인시 거주자가 활동한 지역 데이터

In [ ]:
# In[60]:


def mk_cnt(df,col): ## 앱실행 횟수 모음. 출퇴근 인구는 어떻게 처리할지?
    
    res = df.groupby(col)['count_cust'].sum()
    
    return res

groups = ['dow', 'time_zone','adng_nm']
df_grpby = []

for col in groups:
    df_grpby.append(mk_cnt(df_res, col) + mk_cnt(df_act,col))

In [ ]:
# In[61]:


sns.set_style('darkgrid')
sns.lineplot(data = df_grpby[0]) # 수요일이 제일 많고, 목,금요일 순으로 많음

In [ ]:
# In[25]:

sns.set_style('darkgrid')
sns.lineplot(data = df_grpby[1]) #time zone 4 압도적으로 많음

In [ ]:
# In[26]:


# 상위 10개 동 수요량. 유의미한 차이는 없는듯?
df_top10 = df_grpby[2].sort_values(ascending = False).head(10)
df_top10 = df_top10.reset_index()
sns.barplot(x = 'adng_nm', y = 'count_cust', data = df_top10)

In [ ]:
# In[27]:


df_top10 = df_grpby[2].sort_values(ascending = False).tail(10)
df_top10 = df_top10.reset_index()
sns.barplot(x = 'adng_nm', y = 'count_cust', data = df_top10)

In [ ]:
# In[31]:


youngin_EMD.head()

In [ ]:
# In[102]:


# 일렉트로닉 쇼크
## 결측값이 존재하는데, 동 구분 기준이 약간 다른듯.
## 밑의 지도와 비교해서 emd_cd 직접 수정 필요.


# 한글 폰트 깨짐 방지
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [ ]:
# 마이너스 부호 깨짐 현상 방지
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False


maps = map_

maps['coords'] = maps['geometry'].apply(lambda x: x.representative_point().coords[:])
maps['coords'] = [coords[0] for coords in maps['coords']]

to_be_mapped = 'count_cust'
vmin, vmax = 0,max(maps['count_cust'])
fig, ax = plt.subplots(1, figsize=(30,30))


maps.plot(column=to_be_mapped, cmap='Blues', linewidth=0.8, ax=ax, edgecolors='0.8')
ax.set_title('electronic shock counts', fontdict={'fontsize':30})
ax.set_axis_off()

for idx, row in maps.iterrows():
    plt.annotate(s=row['adng_nm'], xy=row['coords'], horizontalalignment='center', color= 'k')
    
#sns.scatterplot(data = df_res_act,x = 'cell_xcrd', y = 'cell_ycrd',alpha = 0.5,size = 1)

sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []

cbar = fig.colorbar(sm, orientation='horizontal')
plt.savefig(path + 'output/plot/app_map.png')